In [1]:
!pip install bs4

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=43c7a251f2b26ddc4e044f2a553fae74f0458d6904d1a859ff758773caaa6a5e
  Stored in directory: c:\users\monod\appdata\local\pip\cache\wheels\73\2b\cb\099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built bs4


In [2]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install requests

Defaulting to user installation because normal site-packages is not writeable


In [58]:
from bs4 import BeautifulSoup
import requests
import time

url = "https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_1"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    'Accept-Language': 'en-US, en;q=0.5'
}

def scrape_website(url, headers):
    max_retries = 5
    retry_delay = 2
    
    for i in range(max_retries):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                print("OK")
                soup = BeautifulSoup(response.content, 'html.parser')
               
                return
            else:
                print(f"Request failed with status code: {response.status_code}")
        except requests.exceptions.RequestException as e:
            print(f"Error: {e}")
        
        # Retry after delay
        print("Retryin")
        time.sleep(retry_delay)
    
    print("Null")

# Call the function to start scraping
scrape_website(url, headers)


Request failed with status code: 503
Retryin
Request failed with status code: 503
Retryin
Request failed with status code: 503
Retryin
Request failed with status code: 503
Retryin
Request failed with status code: 503
Retryin
Null


In [ ]:
# ASSIGNMENT 1

In [66]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

def scrape_product_details(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
        'Accept-Language': 'en-US, en;q=0.5'
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    product_urls = []
    product_names = []
    product_prices = []
    product_ratings = []
    product_reviews = []
    product_asin = []

    #product_list = soup.find_all('div', {'class': 's-card-container s-overflow-hidden aok-relative puis-wide-grid-style puis-wide-grid-style-t1 puis-include-content-margin puis puis-'}) # tried a couple of times but I received Null Data
    product_list = soup.find_all('div', {'data-component-type': 's-search-result'}) # took help from another source
    for product in product_list:
        url_elem = product.find('a', {'class': 'a-link-normal'})
        url = 'https://www.amazon.in' + url_elem['href'] if url_elem else None
        product_urls.append(url)
        
        name_elem = product.find('span', {'class': 'a-size-medium a-color-base a-text-normal'})
        name = name_elem.text.strip() if name_elem else None
        product_names.append(name)
        
        price_elem = product.find('span', {'class': 'a-price-whole'})
        price = price_elem.text.replace(',', '') if price_elem else None
        price = price.replace('M.R.P: ', '') if price else None
        product_prices.append(float(price) if price else None)
        
        rating_elem = product.find('span', {'class': 'a-icon-alt'})
        rating = rating_elem.text.split()[0] if rating_elem else None
        product_ratings.append(float(rating) if rating else None)
        
        reviews_elem = product.find('span', {'class': 'a-size-base'})
        reviews = reviews_elem.text.replace(',', '') if reviews_elem else None
        reviews = re.sub(r'\D', '', reviews) if reviews else None
        product_reviews.append(int(reviews) if reviews else None)
        

    data = {
        'Product URL': product_urls,
        'Product Name': product_names,
        'Product Price': product_prices,
        'Rating': product_ratings,
        'Number of Reviews': product_reviews,
    }

    df = pd.DataFrame(data)
    return df

def scrape_multiple_pages(base_url, num_pages):
    all_data = []
    for page in range(1, num_pages + 1):
        url = base_url + f'&page={page}'
        data = scrape_product_details(url)
        all_data.append(data)
    
    result = pd.concat(all_data, ignore_index=True)
    return result

base_url = 'https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_1'
num_pages = 30

result = scrape_multiple_pages(base_url, num_pages)
print(result)
result.to_csv('Assignment 1.csv', index=False)



                                           Product URL  \
0    https://www.amazon.in/POLESTAR-Vintage-Laptop-...   
1    https://www.amazon.in/ADISA-BP005-Weight-Casua...   
2    https://www.amazon.in/Safari-Laptop-Backpack-R...   
3    https://www.amazon.in/Safari-Laptop-Backpack-C...   
4    https://www.amazon.in/gp/bestsellers/luggage/2...   
..                                                 ...   
157  https://www.amazon.in/Wildcraft-Ltrs-orange-Ru...   
158  https://www.amazon.in/Puma-Unisex-Adult-Backpa...   
159  https://www.amazon.in/PUMA-Laptop-Backpack-CAS...   
160  https://www.amazon.in/Skybags-Unisex-Black-Fab...   
161  https://www.amazon.in/Genie-Clara-Backpack-Wom...   

                                          Product Name  Product Price  Rating  \
0    POLESTAR Vintage 15.6 inch 32 LTR Casual Lapto...          549.0     3.9   
1           ADISA Light Weight Laptop Backpack 32 Ltrs          499.0     4.0   
2    Safari Quill Laptop Backpack 26 Ltrs Water Res...      